In [16]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import StratifiedKFold
from scipy.spatial.distance import euclidean
from sklearn.neighbors import KernelDensity
import sys; sys.path.append('../Utils')
from utils import Utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
ut = Utils()
X, Y = ut.get_matrixes()

In [18]:
def train_kernel(X, Y, hs, kernels):
    Folds = 4
    skf = StratifiedKFold(n_splits=Folds)
    f1 = []
    gmean = []
    eficiencia_train = []
    eficiencia_test = []
    for kernel in kernels:
        print("kernel: " + str(kernel) + "\n")
        for bandwidth in hs:
            print("bandwidth: " + str(bandwidth) + "\n")
    
            for train, test in skf.split(X, Y):
                X_train, Y_train, X_test, Y_test = ut.get_training_test(X=X, Y=Y, train=train, test=test)
                kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(X)
                f1_score_current, gmean_current, eficiencia_train_current, eficiencia_test_current = ut.get_kernel_metrics(model=kde, X_train=X_train, X_test=X_test, y_train=Y_train, Ytest=Y_test)
                f1.append(f1_score_current)
                gmean.append(gmean_current)
                eficiencia_train.append(eficiencia_train_current)
                eficiencia_test.append(eficiencia_test_current)
    return f1_mean, f1_ic, gmean_mean, gmean_ic, eficiencia_train_mean, eficiencia_train_ic, eficiencia_test_mean, eficiencia_test_ic

In [19]:
hs = [0.05, 0.1, 0.5, 1, 2, 5, 10]
kernels = ['gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine']
f1_mean, f1_ic, gmean_mean, gmean_ic, eficiencia_train_mean, eficiencia_train_ic, eficiencia_test_mean, eficiencia_test_ic = train_kernel(X,Y, hs, kernels)


kernel: gaussian

bandwidth: 0.05



ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [ ]:
hs = [0.05, 0.1, 0.5, 1, 2, 5, 10]
kernels = ['gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine']
f1_mean, f1_ic, gmean_mean, gmean_ic, eficiencia_train_mean, eficiencia_train_ic, eficiencia_test_mean, eficiencia_test_ic = train_kernel(X,Y, hs, kernels)


In [ ]:
df_types = pd.DataFrame({'kernel':pd.Series(['gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian', 'gaussian',
                                             'tophat', 'tophat', 'tophat', 'tophat', 'tophat', 'tophat',  'tophat',
                                             'epanechnikov', 'epanechnikov', 'epanechnikov', 'epanechnikov', 'epanechnikov', 'epanechnikov', 'epanechnikov',
                                             'exponential', 'exponential','exponential', 'exponential', 'exponential', 'exponential','exponential', 
                                             'linear', 'linear', 'linear', 'linear', 'linear', 'linear','linear',
                                             'cosine', 'cosine', 'cosine', 'cosine','cosine', 'cosine', 'cosine']),
                         'C' : pd.Series([0.05, 0.1, 0.5, 1, 2, 5, 10, 0.05, 0.1, 0.5, 1, 2, 5, 10, 0.05, 0.1, 0.5, 1, 2, 5, 10,
                                          0.05, 0.1, 0.5, 1, 2, 5, 10, 0.05, 0.1, 0.5, 1, 2, 5, 10, 0.05, 0.1, 0.5, 1, 2, 5, 10])})

df_types["F1_Score"] = ""
df_types["F1_Score IC"] = ""
df_types["Gmean"]= ""
df_types["Gmean IC"]= ""
df_types["Eficiencia Train"] = ""
df_types["Eficiencia Train IC"] = ""
df_types["Eficiencia Test"] = ""
df_types["Eficiencia Test IC"] = ""

In [ ]:
plot_confusion_matrix(clf,X=X_test, y_true=y_test,normalize='true')

In [ ]:
Yest = clf.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test, Yest, pos_label=2)
roc_auc = auc(fpr, tpr)

In [ ]:
y = label_binarize(y, classes=[1, 2, 3])
n_classes = y.shape[1]
n_classes

In [ ]:
y_test = np.matrix(y_test)
y_test

In [ ]:
y_score.shape

In [ ]:
y_test = y_test.T
y_test.shape

In [ ]:
y_test.shape

In [ ]:
y_pred2 = clf.predict_log_proba(X_test)
score = y_pred2[:,0]-y_pred2[:,1]
tpr,fpr,_ = roc_curve(Y, score)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y_test[:, ]

In [ ]:
 y_score[:, 0]

In [ ]:
df_types